<a href="https://colab.research.google.com/github/SaurabhSinha99/Gemma_Finetune/blob/main/Gemma_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**QLoRA Finetuning of Gemma-2B-IT for Quote Author Identification**

In [ ]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

**We are using gemma-2b-it model. We will quantize the model to 4bit and run some query to check how model is performing.**



In [ ]:
model_id = "google/gemma-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


In [ ]:
os.environ["HF_TOKEN"] = userdata.get('HUGGINGFACE_TOKEN')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
device = "cuda:0"

texts = [
    "Who is the author of the quote: “Be yourself; everyone else is already taken.”?",
    "Who is the author of the quote: “I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”?",
    "Who is the author of the quote: 'So many books, so little time.'?"
]

for text in texts:
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=200)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("-" * 200)  # Separation line

Who is the author of the quote: “Be yourself; everyone else is already taken.”?

The quote is often attributed to Coco Chanel, but there is no evidence to support this claim.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Who is the author of the quote: “I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”?

I cannot find any information about the author of this quote, so I cannot answer this question from the provided context.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Who is the author of the quote: 'So many books, so little time

**The answer for the first question should be Oscar Wilde, for second it should be Marilyn Monroe and for third it should be Frank Zappa, according to this dataset** https://huggingface.co/datasets/Abirate/english_quotes/viewer/default/train . **Moreover the answers are not coming in the format that we would like.**

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [ ]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")

README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")

In [ ]:
def formatting_func(example):
    text = f"Question: Who is the author of the quote: '{example['quote']}'?\nAnswer: {example['author']}"
    return text

**Testing our formatting_func**



In [ ]:
for i in range(5):
  print(formatting_func(data['train'][i]))
  print("-" * 50)

Question: Who is the author of the quote: '“Be yourself; everyone else is already taken.”'?
Answer: Oscar Wilde
--------------------------------------------------
Question: Who is the author of the quote: '“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”'?
Answer: Marilyn Monroe
--------------------------------------------------
Question: Who is the author of the quote: '“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”'?
Answer: Albert Einstein
--------------------------------------------------
Question: Who is the author of the quote: '“So many books, so little time.”'?
Answer: Frank Zappa
--------------------------------------------------
Question: Who is the author of the quote: '“A room without books is like a body without a soul.”'?
Answer: Marcus Tullius Cicero
-----------------

**We would like our answers to be returned in the above format.**

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=625,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=25,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        report_to="none",
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss
25,3.381000
50,2.038200
75,1.786400
100,1.904500
125,1.869500
150,1.942200
175,1.826200
200,1.822600
225,1.674100
250,1.831400


TrainOutput(global_step=625, training_loss=1.8501396850585938, metrics={'train_runtime': 643.9392, 'train_samples_per_second': 3.882, 'train_steps_per_second': 0.971, 'total_flos': 1803001490042880.0, 'train_loss': 1.8501396850585938})

**We will run the above queries again.**

In [ ]:
device = "cuda:0"

texts = [
    "Who is the author of the quote: “Be yourself; everyone else is already taken.”?",
    "Who is the author of the quote: “I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”?",
    "Who is the author of the quote: 'So many books, so little time.'?"
]

for text in texts:
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=200)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("-" * 200)

Who is the author of the quote: “Be yourself; everyone else is already taken.”?
Answer: Oscar Wilde
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Who is the author of the quote: “I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”?
Answer: Marilyn Monroe
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Who is the author of the quote: 'So many books, so little time.'?
Answer: J.K. Rowling,
-----------------------------------------------------------------------------------------------------------------------------------------------------

**The answers to the first and second question are correct and the answer to the third question is still not correct as per the dataset we have used** **but** **all the answers are coming in the desired format.**